## Connect-Four (“4 em Linha”) 

Trabalho realizado por:
- Ana Francisca Rocha  
- Cátia Correia        
- Isabel Sá            

### Índice

1. [Introdução](#Introdução)
2. [Definição de Parâmetros para o Connect Four](#definição-de-parâmetros-para-o-connect-four)
3. [A* Algorithm](#a-algorithm)
    - 3.1 [Heurística do A*](#heurística-do-a-algorithm)
4. [Minimax](#minimax)
5. [Monte Carlo Tree Search (MCTS)](#monte-carlo-tree-search-mcts)
6. [Implementação do Connect Four](#implementação-do-connect-four)
7. [Testes realizados](#testes-realizados)
8. [Bibliografia](#bibliografia)

### Introdução

O presente trabalho incide sobre estratégias de pesquisa informada e adversarial, no âmbito da Inteligência Artificial, mais concretamente no jogo "Connect Four".
Os jogos com adversário são um campo importante da inteligência artificial, que têm sido estudados desde a década de 1950. Estes jogos envolvem dois jogadores, que tomam decisões alternadas para alcançar um objetivo final. Um dos jogos mais conhecidos é o 4 em linha, um jogo de estratégia, em que os jogadores tentam obter quatro peças da mesma cor em uma linha.

Existem várias abordagens algorítmicas para resolver jogos adversariais. Neste trabalho, abordamos três algoritmos diferentes: o algoritmo A*, Minimax e Monte Carlo Tree Search (MCTS), aplicados ao jogo de 4 em linha.

O objetivo deste trabalho é implementar um algoritmo capaz de jogar Connect Four contra um oponente humano. A metodologia utilizada baseia-se na pesquisa bibliográfica, complementada pela colaboração entre os membros do grupo.

### Definição de Parâmetros para o Connect four

In [1]:
import math
import random

MCTS_ITERATIONS = 10000
EXPLORATION = math.sqrt(2)
DEFEAT_SCORE = -1
VICTORY_SCORE = 1
DRAW_SCORE = 0
INI_RUNS = 5
MAX_DEPTH = 3

INF = float('inf')

NUM_ROWS = 6
NUM_COLS = 7
DESL_MAX = 3
LIM_ESQ_MAT = 0
LIM_DRT_MAT = 6
LIM_SUP_MAT = 0
LIM_INF_MAT = 5

### A* Algorithm

O A* é um algoritmo de pesquisa, que consiste numa técnica para encontrar o caminho mais curto entre um nó inicial (fonte) e um nó alvo (destino), num grafo ponderado. Este considera, não apenas o custo real do nó inicial até ao nó atual (g), mas também estima o custo do nó atual ao nó alvo, usando uma heurística (h). Em seguida, seleciona o nó com o menor valor de f (f = g + h) para ser o próximo nó a ser explorado, até que o nó alvo seja alcançado. 

<p align="center">
  <img src="https://adacomputerscience.org/images/content/computer_science/data_structures_and_algorithms/pathfinding/figures/ada_cs_path_star_weighted_graph.png" height="250px" alt="Algoritmo de busca A*">
</p>

<p align="center">Figura nº 1: Algoritmo de busca A*</p>

Informações retiradas e traduzidas do [site OpenGenus IQ](https://iq.opengenus.org/a-search/)

Vantagens:
- É o algoritmo de pesquisa ótimo em termos de heurística.
- É uma das melhores técnicas de pesquisa heurística.
- É utilizado para resolver problemas de pesquisa complexos.

Desvantagens:
- Este algoritmo é completo se o fator de ramificação for finito e cada ação tiver um custo fixo.
- O desempenho da pesquisa A* depende da exatidão do algoritmo heurístico utilizado para calcular a função h(n). 
- Neste problema, tem em conta o tabuleiro atual, não considerando as jogadas futuras do adversário


#### Heurística do A*

A heurística utilizada para o jogo dos 4 em linha foi a utilidade de cada tabuleiro. Tendo como base a seguinte tabela:

| Configuração           | Valor |
|------------------------|-------|
| Vitória por X          | +512  |
| Vitória por O          | -512  |
| Empate ou mistura de X's/O's | 0 |
| Três Os no segmento       | -50   |
| Dois Os no segmento     | -10   |
| Um O isolado no segmento          | -1    |
| Um X isolado no segmento          | +1    |
| Dois Xs no segmento       | +10   |
| Três Xs no segmento       | +50   |
| Bónus de movimento (X) | +16   |
| Bónus de movimento (O) | -16   |

O código implementado:

In [2]:
class AStar:
    def __init__(self, game):
        self.game = game

    def ai_move(self):
        max_i = 0
        points = [0] * len(self.game.available_moves)
        for col in range(len(self.game.available_moves)):
            points[col] = self.check_spot(self.game.available_moves[col], self.game.player)
            if game.player == 'X' and points[col] > points[max_i]:
                max_i = col
            elif game.player == 'O' and points[col] < points[max_i]:
                max_i = col
        return self.game.available_moves[max_i]

    def check_spot(self, col, player):  #calculate the points a spot gets by adding the points of the four consecutive spots, includind the current one, in the eight possible directions when valid
        if player == 'X':
            points = 16
        elif player == 'O':
            points = -16
        desl = -DESL_MAX
        row = self.game.column_heights[col]

        while (desl <= 0):
            if (col + desl >= LIM_ESQ_MAT and col + desl + DESL_MAX <= LIM_DRT_MAT):
                points += self.calc_line(col + desl, row, 1, 0, player)                            #horizontal
                if (row - desl <= LIM_INF_MAT and row - desl - DESL_MAX >= LIM_SUP_MAT):
                    points += self.calc_line(col + desl, row - desl, 1, -1, player)                #diagonal /
                if (row + desl >= LIM_SUP_MAT and row + desl + DESL_MAX <= LIM_INF_MAT):
                    points += self.calc_line(col + desl, row + desl, 1, 1, player)                 #diagonal \
            if (row + desl >= LIM_SUP_MAT and row + desl + DESL_MAX <= LIM_INF_MAT):
                points += self.calc_line(col, row + desl, 0, 1, player)                            #vertical
            desl += 1
        return points
    
    def calc_line(self, col, row, incx, incy, player):
        n = 4
        final_points = 0
        if (player == 'X'):
            player_x = 1
            player_o = 0
        else:
            player_x = 0
            player_o = 1

        while (n > 0):                     #count how many X's and O's there are
            if (self.game.board[row][col] == 'X'):
                player_x += 1
            elif (self.game.board[row][col] == 'O'):
                player_o += 1
            n -= 1
            row += incy
            col += incx
        
        if (player_x == 0):
            if (player_o == 4):
                final_points -= 512
            elif (player_o == 3):
                final_points -= 50
            elif (player_o == 2):
                final_points -= 10
            elif (player_o == 1):
                final_points -= 1
        elif (player_o == 0):
            if (player_x == 4):
                final_points += 512
            elif (player_x == 3):
                final_points += 50
            elif (player_x == 2):
                final_points += 10
            elif (player_x == 1):
                final_points += 1
        return final_points

### Minimax

O algoritmo Minimax é um algoritmo recursivo utilizado em tomadas de decisão e teoria dos jogos, especialmente em jogos de IA. Ele fornece movimentos ótimos para o jogador, assumindo que o oponente também está a jogar de forma ótima. Por exemplo, considerando dois oponentes: Max e Min. Max tentará maximizar o valor, enquanto Min escolherá o valor mínimo possível. O algoritmo realiza uma pesquisa em profundidade (DFS), o que significa que explorará toda a árvore de jogo o mais profundamente possível, até aos nós folha. 

Informações retiradas e traduzidas do [site Wikipedia](https://simple.wikipedia.org/wiki/Minimax#Advantages)

Vantagens:
- Este algoritmo é completamente imbatível porque calcula todas as combinações possíveis de movimentos em um jogo. Como resultado, ao jogar contra um algoritmo minimax, o computador ganha ou empata.
- Os algoritmos da Minimax são mais precisos do que as redes neurais.
- É muito simples de projetar através da codificação.
- Este algoritmo usa a velocidade de processamento do computador e memória de forma eficiente, por isso é potencialmente melhor do que os seres humanos em jogar.

Desvantagens:
- É extremamente lento para grandes jogos como o xadrez, à medida que o número possível de combinações aumenta.
- Não é possível jogar todos os tipos de jogos. Por exemplo, o algoritmo Minimax não pode jogar Rock, Paper, Scissors.

O código implementado foi o seguinte:

In [3]:
class Minimax:
    def __init__(self, game, player_order):
        self.game = game   
        self.player_order = player_order   

    def ai_move(self):
        if self.game.get_player_name() == 'X' and self.player_order == '1':
            score, best_move = self.minimax(self.game, depth=MAX_DEPTH, maximizing_player=True)  
        elif self.game.get_player_name() == 'O' and self.player_order == '1':
            score, best_move = self.minimax(self.game, depth=MAX_DEPTH, maximizing_player=False)  
        elif self.game.get_player_name() == 'X' and self.player_order == '2':
            score, best_move = self.minimax(self.game, depth=MAX_DEPTH, maximizing_player=False) 
        else:
            score, best_move = self.minimax(self.game, depth=MAX_DEPTH, maximizing_player=True) 
        return best_move

    def minimax(self, game, depth, maximizing_player):
        if (depth == 0 or game.is_finished()) and self.player_order == '1':
            return game.get_score(1), None
        if (depth == 0 or game.is_finished()) and self.player_order == '2':
            return game.get_score(2), None
        
        if maximizing_player:
            max_eval = float('-inf')
            best_move = None
            for move in game.get_valid_moves():
                new_game = game.copy()
                new_game.move(move)
                eval, _ = self.minimax(new_game, depth - 1, False)
                if eval > max_eval:
                    max_eval = eval
                    best_move = move
            return max_eval, best_move
        else:
            min_eval = float('inf')
            best_move = None
            for move in game.get_valid_moves():
                new_game = game.copy()
                new_game.move(move)
                eval, _ = self.minimax(new_game, depth - 1, True)
                if eval < min_eval:
                    min_eval = eval
                    best_move = move
            return min_eval, best_move

A heurística implementada está implementada na lógica do jogo, sendo semelhante à lógica da heurística do A*, diferindo, porque a heurística do minimax avalia o tabuleiro inteiro e atribui pontos com base nos padrões de peças encontrados em todas as direções, enquanto que a do A* está mais focada em avaliar uma posição específica no tabuleiro e considerar as peças ao redor dessa posição.

### Monte Carlo Tree Search (MCTS)

A Pesquisa em Árvore de Monte Carlo constrói uma árvore de pesquisa com n nós, sendo cada nó anotado com a contagem de vitórias e a contagem de visitas. O Monte Carlo consiste em quatro passos essenciais.
<p align="center">
  <img src="https://media.geeksforgeeks.org/wp-content/uploads/mcts_own.png" height="250px"  >
</p>

<p align="center">Figura nº 2 - Árvore do Monte Carlo</p>

Informações retiradas e traduzidas do [site Indeed](https://ca.indeed.com/career-advice/career-development/monte-carlo-simulation)

Vantagens:
- **Representação gráfica**: Uma das vantagens desta simulação é o facto de criar gráficos que podem ser utilizados para explicar os resultados aos analistas internos e externos e às partes interessadas. Isso significa que é mais fácil de entender para profissionais que não trabalham com variáveis e probabilidade com frequência.
- **Variáveis extensas**: A simulação utiliza muitas variáveis e efectua os seus cálculos centenas e milhares de vezes, o que significa que pode ter uma elevada precisão com base nos dados introduzidos na simulação.
- **Causas e efeitos**: A simulação pode dar aos analistas uma melhor compreensão das causas e efeitos das variáveis nos dados. Isto deve-se ao facto de a simulação permitir a manipulação de variáveis simples ou múltiplas e tornarpossível produzir os dados de que os analistas profissionais necessitam para o seu trabalho.

Desvantagens:
- **Usa dados históricos**: Esta simulação utiliza dados históricos para fazer as suas previsões. A simulação pode tornar-se imprecisa se os dados históricos variarem muito em comparação com as condições atuais.
- **Usa valores médios**: Embora a simulação use muitas variáveis, ela geralmente cria médias e usa-as para determinar quais ocorrências são mais prováveis. Embora isso possa fazer com que a simulação funcione rapidamente, também pode causar alguma imprecisão nos resultados, especialmente se os dados variarem entre extremos.

Código implementado:

In [4]:
class MonteCarloNode:
    def __init__(self, game, parent=None):
        self.game = game
        self.parent = parent
        self.children = {}
        self.visits = 0
        self.advscore = 0
        self.score = 0

    def getnumchildren(self):
        return len(self.children)

    def ucb(self, root_player):
        if self.visits == 0:
            return INF
        
        if root_player == self.game.player:
            return self.advscore / self.visits + EXPLORATION * math.sqrt(math.log(self.parent.visits) / self.visits)
        
        return self.score / self.visits + EXPLORATION * math.sqrt(math.log(self.parent.visits) / self.visits)

    def select(self, root_player):
        if not self.children:
            return self

        return max(self.children.values(), key=lambda node: node.ucb(root_player)).select(root_player)

    def expand(self):
        valid_moves = self.game.get_valid_moves()
        for move in valid_moves:
            new_game = self.game.copy()
            new_game.move(move)
            self.children[move] = MonteCarloNode(new_game, parent=self)
        return self.children[random.choice(valid_moves)]

    def simulate(self, current_player):
        temp_game = self.game.copy()
        while not temp_game.is_finished():
            valid_moves = temp_game.get_valid_moves()
            move = random.choice(valid_moves)
            temp_game.move(move)
        return temp_game.get_player_score(current_player)


    def backpropagate(self, root_player, score):
        self.visits += 1
        if score == VICTORY_SCORE:
            self.score += score
        elif score == DEFEAT_SCORE:
            self.advscore -= score

        if self.parent:
            self.parent.backpropagate(root_player, score)
    
class MonteCarlo:
    def __init__(self, game, simulations=MCTS_ITERATIONS):
        self.game = game
        self.player = game.player
        self.root = MonteCarloNode(game)
        self.simulations = simulations

    def algorithm(self, node):
            selected_node = self.select_node(node)
            expanded_node = self.expand_node(selected_node)
            score = self.simulate_node(expanded_node)
            self.backpropagate_node(expanded_node, score)

    def ai_move(self):
        self.algorithm(self.root)
        idx = 1

        list_of_children_values = list(self.root.children.values())

        for idx in range(self.root.getnumchildren() * INI_RUNS - 1):
            if (idx > self.simulations):
                break
            self.algorithm(list_of_children_values[idx % self.root.getnumchildren()])

        for _ in range(idx, self.simulations):
            self.algorithm(self.root)
        best_move = self.select_best_move(self.root)
        
        return best_move

    def select_node(self, node):
        while not node.game.is_finished() and node.children:
            node = node.select(self.player)
        return node

    def expand_node(self, node):
        if not node.game.is_finished() and not node.children:
            return node.expand()
        return node

    def simulate_node(self, node):
        return node.simulate(self.player)

    def backpropagate_node(self, node, score):
        node.backpropagate(self.player, score)

    def select_best_move(self, node):
        best_visits = -float('inf')
        best_move = None
        for move, child in node.children.items():
            if move in node.game.get_valid_moves() and child.visits > best_visits:
                best_visits = child.visits
                best_move = move
        return best_move

    def print_children(self, node):
        for idx, child in node.children.items():
            print(str(idx) + ":" + str(child.score))

### Implementação do Connect four

Código implementado para o jogador humano:

In [5]:
class Manual:
    def __init__(self, game):
        self.player = game.get_player_name()

    def ai_move(self):
        valid_move = False
        while not valid_move:
            try:
                column = int(input("Player " + self.player + " please make a move by choosing your coordinates to play (1-7): ")) - 1
                if 0 <= column <= 6:
                    print("Selected column:", column)
                    return column  
                else:
                    print("Invalid column number. Please choose a number between 1 and 7.")
            except ValueError:
                print("Invalid input. Please enter a number.")

Código implementado para a lógica do jogo:

In [6]:
class ConnectFour:
    def __init__(self):
        self.board = [['-' for _ in range(NUM_COLS)] for _ in range(NUM_ROWS)]
        self.column_heights = [5, 5, 5, 5, 5, 5, 5]
        self.available_moves = list(range(NUM_COLS))
        self.player = None
        self.winner = -1
    
    def print_board(self):
        for row in self.board:
            print(' '.join(row))
        print("1 2 3 4 5 6 7")

    def move(self, column):
        height = self.column_heights[column]
        self.board[height][column] = self.player 
        self.last_move = column 
        if height == 0:
            if column in self.available_moves:  # Check if column is in available_moves before removing
                self.available_moves.remove(column)
            else:
                print("Invalid move:", column, "is not in available moves:", self.available_moves)
        else:
            self.column_heights[column] = height - 1

        self.check_winner()
        self.switch_player()

        return self

    def get_player_name(self):
        if self.player == 'X':
            return ("X")
        elif self.player == 'O':
            return ("O")

    def switch_player(self):
        if self.player == 'X':
            self.player = 'O'
        else:
            self.player = 'X'

    def get_player_score(self, player):
        if self.winner == -1:
            return DRAW_SCORE
        elif player != self.winner:
            return DEFEAT_SCORE
        return VICTORY_SCORE

    def check_winner(self):
        for row in range(6):
            for col in range(4):
                if self.board[row][col] != '-' and self.board[row][col] == self.board[row][col + 1] == \
                        self.board[row][col + 2] == self.board[row][col + 3]:
                    self.winner = self.player
                    return True
        for row in range(3):
            for col in range(7):
                if self.board[row][col] != '-' and self.board[row][col] == self.board[row + 1][col] == \
                        self.board[row + 2][col] == self.board[row + 3][col]:
                    self.winner = self.player
                    return True
        for row in range(3):
            for col in range(4):
                if self.board[row][col] != '-' and self.board[row][col] == self.board[row + 1][col + 1] == \
                        self.board[row + 2][col + 2] == self.board[row + 3][col + 3]:
                    self.winner = self.player
                    return True
        for row in range(3):
            for col in range(6, 2, -1):
                if self.board[row][col] != '-' and self.board[row][col] == self.board[row + 1][col - 1] == \
                        self.board[row + 2][col - 2] == self.board[row + 3][col - 3]:
                    self.winner = self.player
                    return True
        return False

    def is_board_full(self):
        for row in self.board:
            for cell in row:
                if cell == '-':
                    return False
        return True
    
    def is_finished(self):
        return self.winner != -1 or self.is_board_full()

    def copy(self):
        copied_game = ConnectFour()
        copied_game.board = [row[:] for row in self.board]
        copied_game.column_heights = self.column_heights[:]
        copied_game.available_moves = self.available_moves[:]
        copied_game.player = self.player
        copied_game.winner = self.winner
        return copied_game

    def get_valid_moves(self):
        return self.available_moves

    def get_score(self, player):
        def evaluate_segment(segm):
            if self.winner == "X":
                return 512
            elif self.winner == "O":
                return -512

            elif "O" in segm and "X" not in segm:
                if segm.count("O") == 3:
                    return -50
                elif segm.count("O") == 2:
                    return -10
                elif segm.count("O") == 1:
                    return -1
            elif segm.count("-") == 4:
                return 0
            elif "X" in segm and "O" not in segm:
                if segm.count("X") == 1:
                    return 1
                elif segm.count("X") == 2:
                    return 10
                elif segm.count("X") == 3:
                    return 50
            return 0

        # Evaluate all possible straight segments
        self.score = 0
        for i in range(6):
            for j in range(4):
                if j + 3 < 7:  # check if the indices are within range
                    segment = [self.board[i][j + k] for k in range(4)]
                    self.score += evaluate_segment(segment)
        for i in range(4):
            for j in range(7):
                if i + 3 < 6:  # check if the indices are within range
                    segment = [self.board[i + k][j] for k in range(4)]
                    self.score += evaluate_segment(segment)
        for i in range(3):
            for j in range(4):
                if i + 3 < 6 and j + 3 < 7:  # check if the indices are within range
                    segment = [self.board[i + k][j + k] for k in range(4)]
                    self.score += evaluate_segment(segment)
        for i in range(3):
            for j in range(3, 7):
                if i + 3 < 6 and j - 3 >= 0:  # check if the indices are within range
                    segment = [self.board[i + k][j - k] for k in range(4)]
                    self.score += evaluate_segment(segment)
        if player == 1:
            return self.score
        else:
            return -(self.score)
    
    
    def play(self):
        player_x_wins = 0
        player_o_wins = 0
        ties = 0
        print("Welcome to Connect Four!")

        game_choice = input("1 for Start Game | 2 for Cancel: ").strip()
        if game_choice == '2':
            print("Game canceled.")
            return

        player_choices = {}

        player_order = input("Who plays first? (1 for X / 2 for O): ").strip()
        while player_order not in ['1', '2']:
            player_order = input("Invalid choice. Please choose 1 for X or 2 for O: ").strip()

        if player_order == '1':
            player_choices['X'] = input(
                "Choose player 'X' (1 for Manual / 2 for AStar / 3 for MonteCarlo / 4 for Minimax): ").strip()
            player_choices['O'] = input(
                "Choose player 'O' (1 for Manual / 2 for AStar / 3 for MonteCarlo / 4 for Minimax): ").strip()
        else:
            player_choices['O'] = input(
                "Choose player 'O' (1 for Manual / 2 for AStar / 3 for MonteCarlo / 4 for Minimax): ").strip()
            player_choices['X'] = input(
                "Choose player 'X' (1 for Manual / 2 for AStar / 3 for MonteCarlo / 4 for Minimax): ").strip()

        num_games = int(input("How many games do you want to play? "))

        for game_num in range(1, num_games + 1):
            print(f"Game {game_num} out of {num_games}:")

            # Choose player to start the game
            if player_order == '1':
                starting_player = 'X'
            else:
                starting_player = 'O'

            self.__init__()  # Reinicializa o jogo
            self.player = starting_player

            self.print_board()

            while self.winner == -1 and not self.is_board_full():
                print("It is now {}'s turn.".format(self.player))
                player_choice = player_choices[self.player]

                if player_choice == '1':  # Manual
                    algorithm = Manual(self)

                elif player_choice == '2':  # AStar
                    algorithm = AStar(self)

                elif player_choice == '3':  # MonteCarlo
                    algorithm = MonteCarlo(self)

                elif player_choice == '4':
                    algorithm = Minimax(self,player_order)
                    
                column = algorithm.ai_move()
                self.move(column)

                self.print_board()

            if self.winner != -1:
                print("Congratulations! {} wins!".format(self.winner))
                if self.winner == 'X':
                    player_x_wins += 1
                elif self.winner == 'O':
                    player_o_wins += 1
            else:
                print("It's a tie!")
                
                ties += 1    
        print("===============================")
        print("Player X", end=" ")
        if player_choices['X'] == '1':
            print("Manual", end=" ")
        elif player_choices['X'] == '2': 
            print("A*", end=" ")
        elif player_choices['X'] == '3':
            print("MonteCarlo", end=" ")
        elif player_choices['X'] == '4':
            print("Minimax", end=" ")
        print("won", player_x_wins, "times")
        print ("Player O " , end=" ")
        if player_choices['O'] == '1':
            print("Manual", end=" ")
        elif player_choices['O'] == '2': 
            print("A*", end=" ")
        elif player_choices['O'] == '3':
            print("MonteCarlo", end=" ")
        elif player_choices['O'] == '4':
            print("Minimax", end=" ")
        print("won", player_o_wins, "times")
        print(f"Ties: {ties} ")
        print("===============================")

Código implementado para correr o jogo:

In [7]:
if __name__ == '__main__':
    game = ConnectFour()
    game.play()

Welcome to Connect Four!
Game canceled.


### Testes realizados

| Configurações iniciais           |
|------------------------ |
| MCTS_ITERATIONS = 10000 |
| EXPLORATION = math.sqrt(2)           |
| DEFEAT_SCORE = -1 |
| VICTORY_SCORE = 1       |
| DRAW_SCORE = 0     |
| MAX_DEPTH = 3          |


Para as configurações iniciais, obtivemos os resultados abaixo apresentados:

| PLAYER X \ PLAYER O   | A STAR          |  MCTS            | MINIMAX    |
------------------------|-----------------|-----------------|------------|
| **A STAR**| <p> X: 100 <br> O: 0 <br> TIES = 0 </p> |  <p> X: 0 <br> O: 100 <br> TIES = 0 </p>  | <p> X: 0 <br> O: 100 <br> TIES = 0 </p>  |
| **MCTS**|   <p> X: 100 <br> O: 0 <br> TIES = 0 </p> |  <p> X: 71 <br> O: 28 <br> TIES = 1 </p>   |    <p> X: 48 <br> O: 51 <br> TIES = 1  </p>   |               
| **MINIMAX**|  <p> X: 100 <br> O: 0 <br> TIES = 0  </p>  | <p> X: 7  <br> O: 93  <br> TIES = 0 </p> |   <p> X: 0 <br> O: 100 <br> TIES = 0 </p> |               

Como esperado, por ser um algoritmo que apenas procura obter a maior pontuação, o A* perde todos os jogos contra ambos os algoritmos adversariais (MCTS e Minimax).

Por outro lado, entre o MCTS e o Minimax, no qual o MCTS é o primeiro a jogar, observamos uma competição bastante equilibrada, com uma margem mínima de vitória a favor do Minimax. No entanto, ao colocar o Minimax como o primeiro jogador, observou-se uma vitória significativa do MCTS.

Para além disso, achamos que seria interessante colocar os algoritmos contra eles mesmos para perceber as vantagens de jogar, ou não, em primeiro. Observamos então que o A star tem mais vantagens ao jogar em primeiro lugar e que o MCTS, apesar de não ter uma vitória de 100 a 0, tem também essa mesma vantagem. Por outro lado, no jogo Minimax vs Minimax, observamos uma derrota de 0 a 100 do primeiro a jogar.

---

Para contrariar a vitória significativa do MCTS nas partidas entre Minimax vs MCTS, realizamos um teste, com MAX_DEPTH = 5:

| Configurações           |
|------------------------ |
| MCTS_ITERATIONS = 10000 |
| EXPLORATION = math.sqrt(2)        |
| DEFEAT_SCORE = -1 |
| VICTORY_SCORE = 1       |
| DRAW_SCORE = 0     |
| MAX_DEPTH = 5          |


Resultados:

|MINIMAX(X) vs MCTS(O) |
----------------------
|X = 80 |
|O = 12 |
|Ties = 8|

Conforme previsto, com o aumento da profundidade do Minimax o algoritmo melhorou significativamente os seus resultados. Por outro lado, o tempo de resposta do algoritmo aumenta também, o que poderá não ser vantajoso.

---

De seguida, na tentativa de aumentar o número de vitórias do MCTS nas partidas entre MCTS vs Minimax, realizamos testes com alterações em diferentes parâmetros, tais como:






- MCTS_ITERATIONS


| Configurações           |
|------------------------ |
| MCTS_ITERATIONS = 20000 |
| EXPLORATION = math.sqrt(2)        |
| DEFEAT_SCORE = -1 |
| VICTORY_SCORE = 1      |
| DRAW_SCORE = 0     |
| MAX_DEPTH = 3          |

Resultados:

|MCTS(X) vs MINIMAX(O) |
----------------------
|X = 74 |
|O = 22 |
|Empates = 4|


---

| Configurações           |
|------------------------ |
| MCTS_ITERATIONS = 30000 |
| EXPLORATION = math.sqrt(2)        |
| DEFEAT_SCORE = -1 |
| VICTORY_SCORE = 1      |
| DRAW_SCORE = 0     |
| MAX_DEPTH = 3          |

Resultados:

|MCTS(X) vs MINIMAX(O) |
----------------------
|X = 76 |
|O = 24 |
|Empates = 0|

Neste caso, ao aumentar as iterações, como esperado, o MCTS melhorou o seu resultado. No entanto, de 20 000 iterações para 30 000 iterações não houve uma melhoria acentuada, pelo que, não existe interesse em utilizá-lo, pois o tempo de resposta é substancialmente superior.

---
- EXPLORATION

| Configurações           |
|------------------------ |
| MCTS_ITERATIONS = 10000 |
| EXPLORATION = 1.1        |
| DEFEAT_SCORE = -1 |
| VICTORY_SCORE = 1      |
| DRAW_SCORE = 0     |
| MAX_DEPTH = 3          |

Resultados:

|MCTS(X) vs MINIMAX(O) |
----------------------
|X = 62 |
|O = 36 |
|Empates = 2|

---

| Configurações           |
|------------------------ |
| MCTS_ITERATIONS = 10000 |
| EXPLORATION = 0.8        |
| DEFEAT_SCORE = -1 |
| VICTORY_SCORE = 1      |
| DRAW_SCORE = 0     |
| MAX_DEPTH = 3          |

Resultados:

|MCTS(X) vs MINIMAX(O) |
----------------------
|X = 81 |
|O = 17 |
|Empates = 2|

EXPLORATION é uma constante que define se o algoritmo tende a explorar menos e a aproveitar mais as ações que parecem ser as melhores, para valores menores, ou tende a explorar mais, ou seja, tenta ações menos escolhidas anteriormente com mais frequência, para ganhar mais informações sobre elas, para valores maiores. 

Os valores escolhidos para os testes acima encontram-se no intervalo sugerido de 0.8 a 1.4, para existir um balanceamento do que irá ser explorado.

Com isto, ao alterar o valor para 1.1, constatamos que o número de vitórias do MCTS aumentou. Já para EXPLORATION = 0.8, o número de vitórias aumentou significativamente.

Estas conclusões estão de acordo com o que esperávamos, visto que o algoritmo está mais focado nas ações que o poderão beneficiar mais, no entanto, podemos estar a deixar de explorar algumas opções que poderiam ser também vantajosas ele.

---

### Bibliografia

Código do minimax - Pseudocódigo presente nos PowerPoints das aulas

Código do monte Carlo inspirado - https://www.harrycodes.com/blog/monte-carlo-tree-search#monte-carlo-tree-search-mcts Acedido em: 07/03/2024
 
Código para a lógica do jogo inspirado num notebook disponibilizado na cadeira de Elementos de Inteligência Artificial e Ciência de Dados, referente ao Connect Four e também foi inspirado pelo seguinte trabalho - [https://- github.com/erikackermann/Connect-Four/blob/master/connect4.py](https://github.com/erikackermann/Connect-Four/blob/master/connect4.py) Acedido em: 05/03/2024

https://ca.indeed.com/career-advice/career-development/monte-carlo-simulation Acedido em: 28/03/2024

https://simple.wikipedia.org/wiki/Minimax#Advantages Acedido em: 28/03/2024

https://iq.opengenus.org/a-search/ Acedido em: 28/03/2024
